In [ ]:
# import nest_asyncio
# nest_asyncio.apply()
# 
# import asyncio
# from slack import RTMClient
# 
# @RTMClient.run_on(event="message")
# def list_message(**payload):
#     data = payload['data']
#     print(data)  # Handle the incoming message data as per your requirements
# 
# with open('slack-api-token.txt', 'r') as file:
#     slack_token = file.read().strip()
# 
# rtm_client = RTMClient(token=slack_token)
# asyncio.ensure_future(rtm_client.start())
# asyncio.get_event_loop().run_forever()

/Users/jmcaffee/git/python-playground-v2/venv/lib/python3.9/site-packages/slack/deprecation.py:14: UserWarning: slack package is deprecated. Please use slack_sdk.web/webhook/rtm package instead. For more info, go to https://slack.dev/python-slack-sdk/v3-migration/
  warnings.warn(message)


In [1]:
# from slack_sdk.rtm_v2 import RTMClient
# with open('slack-api-token.txt', 'r') as file:
#     slack_token = file.read().strip()
# 
# rtm = RTMClient(token=slack_token)
# 
# @rtm.on("message")
# def handle(client: RTMClient, event: dict):
#     if 'Hello' in event['text']:
#         channel_id = event['channel']
#         thread_ts = event['ts']
#         user = event['user'] # This is not username but user ID (the format is either U*** or W***)
# 
#         client.web_client.chat_postMessage(
#             channel=channel_id,
#             text=f"Hi <@{user}>!",
#             thread_ts=thread_ts
#         )
# 
# rtm.connect()

In [2]:
from llama_cpp import Llama
max_tokens = 2048
# llama_llm = Llama(model_path="/Volumes/4Tera_SSD_2022/llm_models/llama-2-7b-chat.Q5_K_M.gguf", chat_format="llama-2")
llm = Llama(model_path="/Volumes/Macintosh HD/Users/jmcaffee/llm-models/llama-2-7b-chat.Q5_K_M.gguf", chat_format="llama-2", n_ctx=max_tokens)
llm.verbose = False

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from /Volumes/Macintosh HD/Users/jmcaffee/llm-models/llama-2-7b-chat.Q5_K_M.gguf (version GGUF V2)
llama_model_loader: - tensor    0:                token_embd.weight q5_K     [  4096, 32000,     1,     1 ]
llama_model_loader: - tensor    1:           blk.0.attn_norm.weight f32      [  4096,     1,     1,     1 ]
llama_model_loader: - tensor    2:            blk.0.ffn_down.weight q6_K     [ 11008,  4096,     1,     1 ]
llama_model_loader: - tensor    3:            blk.0.ffn_gate.weight q5_K     [  4096, 11008,     1,     1 ]
llama_model_loader: - tensor    4:              blk.0.ffn_up.weight q5_K     [  4096, 11008,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_norm.weight f32      [  4096,     1,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.attn_k.weight q5_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    7:         blk.0.attn_output.weight 

In [4]:
import time
def prompt2(question):
    start_time_seconds = time.time()
    output = llm.create_chat_completion(
        max_tokens=max_tokens,
        messages = [
            {
                "role": "system",
                "content": "You are an assistant who perfectly responds to questions with as many details as possible, without making up facts."
            },
            {
                "role": "user",
                "content": question
            }
        ]
    )
    elapsed_time_seconds = time.time() - start_time_seconds
    # print(output)
    answer = output['choices'][0]['message']['content']
    print(f'answer generated in {elapsed_time_seconds} seconds')
    print(f"Q: {question} \nA:{answer}")
    return answer

In [ ]:
import asyncio
import nest_asyncio
nest_asyncio.apply()
from slack_sdk.rtm_v2 import RTMClient

# Load your Slack API token
with open('slack-api-token.txt', 'r') as file:
    slack_token = file.read().strip()

# Initialize the RTMClient
rtm = RTMClient(token=slack_token)

# Define the event handler
@rtm.on("message")
def handle(client: RTMClient, event: dict):
    print(f"event: {event}")
    message_received = event.get('text', '')
    channel_id = event['channel']
    thread_ts = event['ts']
    user = event['user']  # User ID

    
    # Post a message to the channel
    client.web_client.chat_postMessage(
        channel=channel_id,
        text=f"Having llm answer the question: {message_received}",
        thread_ts=thread_ts
    )
    
    answer = prompt2(message_received)
    client.web_client.chat_postMessage(
        channel=channel_id,
        text=f"Answer: {answer}",
        thread_ts=thread_ts
    )
# Running the event loop
asyncio.ensure_future(rtm.start())
asyncio.get_event_loop().run_forever()


event: {'type': 'message', 'channel': 'D069VA1AZ2B', 'text': 'What is a camel?', 'blocks': [{'type': 'rich_text', 'block_id': '471G0', 'elements': [{'type': 'rich_text_section', 'elements': [{'type': 'text', 'text': 'What is a camel?'}]}]}], 'user': 'UEU96U15F', 'client_msg_id': '362d2353-e70a-4fcc-b04a-3cc96db92eeb', 'team': 'TEUMZQQDA', 'source_team': 'TEUMZQQDA', 'user_team': 'TEUMZQQDA', 'suppress_notification': False, 'event_ts': '1702610300.862539', 'ts': '1702610300.862539'}
event: {'type': 'message', 'channel': 'D069VA1AZ2B', 'text': 'What is a camel?', 'blocks': [{'type': 'rich_text', 'block_id': '471G0', 'elements': [{'type': 'rich_text_section', 'elements': [{'type': 'text', 'text': 'What is a camel?'}]}]}], 'user': 'UEU96U15F', 'client_msg_id': '362d2353-e70a-4fcc-b04a-3cc96db92eeb', 'team': 'TEUMZQQDA', 'source_team': 'TEUMZQQDA', 'user_team': 'TEUMZQQDA', 'suppress_notification': False, 'event_ts': '1702610300.862539', 'ts': '1702610300.862539'}
event: {'type': 'message',